In [ ]:
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers  import Convolution2D , ZeroPadding2D
from tensorflow.python.keras.layers  import MaxPooling2D , Activation 
from tensorflow.python.keras.layers  import Flatten , Dropout
from tensorflow.python.keras.layers  import Dense

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
#Defining a CNN with 6 layers
def define_model():
    model = Sequential()

    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    return model


if __name__ == '__main__':
    model = define_model()
    print(model.summary())

In [ ]:
sgd = tf.keras.optimizers.SGD(lr=0.001,momentum=0.9)
model.compile(optimizer=sgd, loss='categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/train', #Replace with path to train set
                                                 target_size = (224 , 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/test', #Replace with path to test set
                                            target_size = (224 , 224),
                                            batch_size = 64,
                                            class_mode = 'categorical')



In [ ]:
classweights = {0:1.0 , 1:1.1 , 2:1.1, 3:9.1} #Custom class weights to account for unbalanced dataset 

In [ ]:
model.fit(training_set, epochs =10,validation_data = test_set , 
          verbose = 1 , class_weight=classweights ,
          validation_steps=1,
          steps_per_epoch=training_set.samples//training_set.batch_size, 
          validation_steps=test_set.samples//test_set.batch_size)